<a href="https://colab.research.google.com/github/Saadman53/Thesis-Human-Action-Recognition/blob/main/Feature_Extraction_using_Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
gdrive_path = "drive/My Drive/Dataset/cutu_277x277.mat"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data

import torchvision
import torchvision.transforms as transforms



import random
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report


from torch.utils.data import DataLoader, ConcatDataset
# from torchvision import transforms
from sklearn.model_selection  import StratifiedKFold

import torch.optim.lr_scheduler as lr_scheduler
%matplotlib inline

import scipy.io as sio
torch.autograd.set_detect_anomaly(True)


from functools import partial
from typing import Any, Optional

import torch
import torch.nn as nn


In [ ]:
dat = sio.loadmat(gdrive_path)

In [ ]:
X = np.resize(np.array(dat['data']),(1165,1,227,227))
y = np.array(dat['label'])[0]

In [ ]:
def alexnet_feature_extraction(X):
  'Convert to 3 channel'
  data = torch.empty((X.shape[0],3,227,227),dtype=torch.float32)
  for i in range(X.shape[0]):
    t = torch.tensor(X[i], dtype = torch.float32).reshape((1,227,227))
    t = t.expand(3,*t.shape[1:])
    data[i] = t
  data = torch.tensor(data)

  'Apply necessary transformations'
  data_transforms = transforms.Compose([
      transforms.Resize((224,224)),
      transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
  ])
  X_transformed = data_transforms(data)

  'Instantiate the model'
  alexnet = torchvision.models.alexnet(pretrained=True)
  alexnet.classifier = torch.nn.Sequential(*list(alexnet.classifier.children())[:-4]) #keeping only the first layer

  'Create the new dataset'
  dataset = torch.empty((X.shape[0],1,4096),dtype=torch.float32)
  for i in range(data.shape[0]):
    batch_data = torch.unsqueeze(X_transformed[i],0)
    dataset[i] =  alexnet(batch_data)
    print(i*100/data.shape[0])
  return dataset


In [ ]:
X = alexnet_feature_extraction(X)

In [ ]:
dic = {'data': X, 'labels': y, 'filename': dat['filename']}
sio.savemat("drive/My Drive/Dataset/alexnet_extracted_4096.mat", dic)